In [ ]:
# learning using a CNN

# CNN for learning!

# learn the states of a double dot
import numpy as np
import tensorflow as tf
import glob
import os
import time

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

# application logic will be added here
def cnn_model_fn(features,labels,mode):
    '''Model function for CNN'''
    #input layer
    input_layer = tf.cast(tf.reshape(features,[-1,30,30,1]),tf.float32)
    
    conv1 = tf.layers.conv2d(inputs=input_layer,
                            filters=32,
                            kernel_size=[5,5],
                            padding="same",
                            activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[5,5],strides=5)
    flat = tf.contrib.layers.flatten(inputs=pool1)
    # dense output layer
    out1 = tf.layers.dense(inputs=flat,units=32,activation=tf.nn.relu)  
    out = tf.layers.dense(inputs=out1,units=4,activation=tf.nn.relu)
    #out = tf.nn.softmax(out_layer)

    loss = None
    train_op = None

    # Calculate loss( for both TRAIN AND EVAL modes)
    if mode != learn.ModeKeys.INFER:
        loss = tf.losses.mean_squared_error(labels,out)

    # Configure the training op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=0.01,
            optimizer="Adam")

    # Generate predictions
    predictions= {
        "prob" : out,
        "states" : (tf.argmax(out,axis=1) - 1),
    }

    # Returna  ModelFnOps object
    return model_fn_lib.ModelFnOps(mode=mode,predictions=predictions,loss=loss, train_op=train_op)
    
def get_train_inputs():
    n_batch = 10
    index = np.random.choice(n_train,n_batch,replace=False)
    inp = []
    oup = []
    for i in index:
        dat = np.load(files[i])
        inp += [dat.item()['current_map']]
        oup += [dat.item()['label']]

    inp = np.array(inp,dtype=np.float32)
    oup = np.array(oup,dtype=np.float32)
    
    x = tf.constant(inp)
    y = tf.constant(oup)
    return x,y

def get_test_inputs():
    inp = []
    oup = []
       
    for file in files[n_train:(n_train+1000)]:
        dat = np.load(file)
        inp += [dat.item()['current_map']]
        oup += [dat.item()['label']]
    
    inp = np.array(inp)
    oup = np.array(oup)
    
    x = tf.constant(inp,dtype=np.float32)
    y = tf.constant(oup,dtype=np.float32)
    return x,y

st = time.time()
# get the data
data_folder_path = "/Users/ssk4/Downloads/data_subimage/"
files = glob.glob(data_folder_path + "*.npy")

# shuffling the files to avoid any single dot bias
import random
random.shuffle(files)
files = files[:100]

n_samples = len(files)
train_sample_ratio = 0.8
n_train = int(train_sample_ratio * n_samples)

print("Total number of samples :",n_samples)
print("Training samples :",n_train)
print("Test samples :",n_samples - n_train)

st2 = time.time()
# create the estimator
dd_classifier = learn.Estimator(model_fn=cnn_model_fn)

# set up logging for predictions
tensors_to_log = {}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

metrics = {
    "accuracy" : learn.MetricSpec(metric_fn=tf.metrics.accuracy, prediction_key="states"),
}

dd_classifier.fit(
    input_fn=get_train_inputs,
    steps=50000,
    monitors=[logging_hook])

print("Training done in",time.time()-st2,"seconds.")

print("Total number of samples :",n_samples)
print("Training samples :",n_train)
print("Test samples :",n_samples - n_train)

# Training accuracy
x,y = get_train_inputs()
sess = tf.Session()
y_argmax = tf.argmax(y,axis=1) - 1
out = sess.run(y_argmax)

pred = []
predictions=dd_classifier.predict(input_fn=get_train_inputs)
for i, p in enumerate(predictions):
  pred += [p['states']]

diff = abs(out - pred)
arr = list(diff).count(0)/len(diff)
print("Train Accuracy",arr)


# Test accuracy
x,y = get_test_inputs()
sess = tf.Session()
y_argmax = tf.argmax(y,axis=1) - 1
out = sess.run(y_argmax)

pred = []
predictions=dd_classifier.predict(input_fn=get_test_inputs)
for i, p in enumerate(predictions):
  pred += [p['states']]

diff = abs(out - pred)
arr = list(diff).count(0)/len(diff)
print("Test Accuracy",arr)

Total number of samples : 100
Training samples : 80
Test samples : 20
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x110d62128>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/hs/nm7qn85519v78y5bbl49h61h001sq3/T/tmpuim159_x/model.ckpt.
INFO:tensorflow:loss = 0.193754, step = 1
INFO:tensorflow:
INFO:tensorflow:global_step/sec: 51.8105
INFO:tensorflow:loss = 0.00175135, step = 101 (1.932 sec)
INFO

In [12]:
def get_test_inputs():
    inp = []
    oup = []
       
    for file in files[n_train:(n_train + 1000)]:
        dat = np.load(file)
        inp += [dat.item()['current_map'].flatten()]
        oup += [dat.item()['label']]
    
    inp = np.array(inp)
    oup = np.array(oup)
    
    x = tf.constant(inp,dtype=np.float32)
    y = tf.constant(oup,dtype=np.float32)
    return x,y

predictions=dd_classifier.predict(input_fn=get_test_inputs)

x,y = get_test_inputs()
sess = tf.Session()
y_argmax = tf.argmax(y,axis=1) - 1
out = sess.run(y_argmax)

pred = []
for i, p in enumerate(predictions):
  pred += [p['states']]

diff = abs(out - pred)
arr = list(diff).count(0)/len(diff)
print("Accuracy",arr)

INFO:tensorflow:Restoring parameters from /var/folders/hs/nm7qn85519v78y5bbl49h61h001sq3/T/tmpix_moxjd/model.ckpt-1376
Accuracy 0.75
